In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import math
import scipy.stats as stats
pd.set_option("display.max_columns", None) # show all columns
pd.set_option("display.max_rows", None) # show all rows
pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

# Where is your data_path
data_path = '/content/drive/My Drive/Evan_Assignment_EC6/'

In [ ]:
# Read Data
df = pd.read_csv(data_path + 'assignment_employee_survey.csv') #, encoding='latin1')

df.head()

### **Kolom Relevan untuk Analisis `job_satisfaction`**
1. `age` – Usia karyawan.  
2. `gender` – Jenis kelamin.  
3. `marital_status` – Status pernikahan.  
4. `job_level` – Level jabatan.  
5. `experience` – Lama pengalaman kerja (tahun).  
6. `dept` – Departemen tempat bekerja.  
7. `emp_type` – Jenis pekerjaan (Full-Time, Contract, Part-Time).  
8. `wlb` – Work-life balance (skala 1–5).  
9. `work_env` – Kualitas lingkungan kerja (skala 1–5).  
10. `salary` – Gaji.  
11. `training_hours_per_year` – Jumlah jam pelatihan per tahun.  
12. `have_ot` – Sering lembur atau tidak.  
13. `commute_distance` – Jarak perjalanan kerja.  
14. `stress` – Tingkat stres kerja (skala 1–5).  
15. `workload` – Beban kerja (skala 1–5).  
16. `job_satisfaction` – Target analisis (skala 1–5).  

In [ ]:
df.isnull().sum()

In [ ]:
len(df.drop_duplicates())/len(df)

### No duplicate, no null values, so no need for data cleaning?

In [ ]:
list_col = list(df.columns)

In [ ]:
for col in list_col:
    missing_count = df[col].isna().sum()
    missing_percentage = (missing_count / len(df)) * 100
    print(f"Column '{col}' Has {missing_count} missing values ({missing_percentage:.2f}%)") # .2f means 2 decimal

### Check outliers

In [ ]:
df.info()

In [ ]:
data = df.copy()

In [ ]:
numerical_df = data.select_dtypes(include='number')
numerical_df.columns
numcols = numerical_df.columns.to_list()

In [ ]:
def check_plot(df, variable):
    # fungsi mengambil kerangka data (df) dan
    # variabel yang diminati sebagai argumen

    # tentukan ukuran gambar
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins=30, kde=True)
    plt.title('Histogram')

    # plot Q-Q
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('Variable quantiles')

    # box plot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')
    plt.savefig(f'Dist of {col}.png')
    plt.show()

In [ ]:
numcols.remove('emp_id')

**Before**

In [ ]:
for col in numcols:
    check_plot(data, col)

In [ ]:
def find_outlier_boundary(df, variable):

    # Mari kita hitung batas luar yang merupakan outlier
    # Rumus IQR: Q3-Q1
    IQR = df[variable].quantile(0.75) - df[variable].quantile(0.25)
    #LB: Q0 = Q1 - 3/2*IQR, UB: Q4 = Q3 + 3/2*IQR
    lower_boundary = df[variable].quantile(0.25) - (IQR * 1.5)
    upper_boundary = df[variable].quantile(0.75) + (IQR * 1.5)

    return upper_boundary, lower_boundary

In [ ]:
filtered_columns_EDA = []

for col in numcols:
    upper_limit, lower_limit = find_outlier_boundary(data, col)

    # Check if there are values outside the boundary
    is_outlier = (data[col] > upper_limit) | (data[col] < lower_limit)

    if is_outlier.any():
        print(f"Column: {col} | Outliers detected | Upper: {upper_limit}, Lower: {lower_limit}")
        data = data[~is_outlier]
        filtered_columns_EDA.append(col)
    else:
        print(f"Column: {col} | No outliers detected. Skipping filtering.")

print("\nColumns that required filtering due to outliers:")
print(filtered_columns_EDA)

**After**

In [ ]:
# adjust the figure size for better readability/after outlier handling
plt.figure(figsize=(12,6))

# plotting
for i in range(0, len(numcols)):
    plt.subplot(1, len(numcols), i+1)
    sns.boxplot(y=data[numcols[i]], color='red')
    plt.tight_layout()

In [ ]:
for col in filtered_columns_EDA:
    check_plot(data, col)

In [ ]:
categorical_cols = data.select_dtypes(include='object')
catcols = categorical_cols.columns.to_list()
catcols

#### Range Value and Unique Values

In [ ]:
for col in data.columns:
    print(f"=== {col} ===")
    if data[col].dtype == 'object':
        print(data[col].value_counts())
    else:
        print(f"Min: {data[col].min()}, Max: {data[col].max()}")
    print("\n")

In [ ]:
for col in data.columns:
    print(f"=== {col} ===")
    print(data[col].unique())
    print("\n")


#### Statistical Summary

In [ ]:
data[numcols].describe()

In [ ]:
# plot heatmap
corr = data[numcols].corr()

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Heatmap Korelasi Antar Variabel Numerik", fontsize=14)
plt.savefig(data_path + 'Heatmap Correlation for People Analytics.png')
plt.show()

In [ ]:
catcols

### Significant Correlations with Job Satisfaction

#### **Positive Correlations**:
- **`wib` (0.12)**: Slight positive correlation.  
- **`work_env` (0.12)**: Better work environment → higher satisfaction.  
- **`sleep_hours` (0.10)**: More sleep → slightly higher satisfaction.  

#### **Negative Correlations**:
- **`workload` (-0.13)**: Strongest negative effect. Higher workload → lower satisfaction.  
- **`stress` (-0.09)**: Higher stress → lower satisfaction (weaker than workload).  

#### **Negligible Correlations** (≈0):
- `age`, `experience`, `hours_per_year`, `team_size`, `num_reports`.  

---

### **Key Takeaways**:
1. **Top detractors**: Reduce `workload` and `stress` for maximum impact.  
2. **Mild boosters**: Improve `work_env`, `sleep_hours`, and `wib` (if actionable).  

### **Actionable Insights**:
-  **Optimize workload distribution**.  
-  **Introduce stress-management programs**.  
-  **Promote sleep hygiene policies**.  
-  **Enhance workplace environment** (e.g., flexibility, team activities).  

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import spearmanr, pearsonr, kendalltau

alpha = 0.05
results = []

# Pastikan emp_id keluar dari numcols
if 'emp_id' in numcols:
    numcols.remove('emp_id')

# --- Numerik vs Numerik ---
for col in numcols:
    if col != 'job_satisfaction':
        # Spearman
        corr, p = spearmanr(data[col], data['job_satisfaction'])
        method = "Spearman"

        # Bisa ganti ke Pearson atau Kendall jika mau:
        # corr, p = pearsonr(data[col], data['job_satisfaction'])
        # method = "Pearson"
        # corr, p = kendalltau(data[col], data['job_satisfaction'])
        # method = "Kendall"

        conclusion = "Berpengaruh signifikan" if p < alpha else "Tidak signifikan"
        results.append({
            "Variable": col,
            "Test": f"{method} Correlation",
            "Statistic": corr,
            "p-value": p,
            "Conclusion": conclusion
        })

# --- Kategorikal vs Numerik ---
for col in catcols:
    groups = [data.loc[data[col] == cat, 'job_satisfaction'].dropna()
              for cat in data[col].unique()]

    if len(groups) > 1:
        # Cek normalitas tiap group (hanya kalau sampel >= 3)
        normal = all(stats.shapiro(g)[1] > alpha for g in groups if len(g) >= 3)

        if len(groups) == 2:
            if normal:
                stat, p = stats.ttest_ind(groups[0], groups[1])
                test_type = "T-Test"
            else:
                stat, p = stats.mannwhitneyu(groups[0], groups[1])
                test_type = "Mann–Whitney"
        else:
            if normal:
                stat, p = stats.f_oneway(*groups)
                test_type = "ANOVA"
            else:
                stat, p = stats.kruskal(*groups)
                test_type = "Kruskal–Wallis"

        results.append({
            'Variable': col,
            'Test': test_type,
            'Statistic': stat,
            'p-value': p,
            'Conclusion': 'Berpengaruh signifikan' if p < alpha else 'Tidak signifikan'
        })

# DataFrame hasil
results_df = pd.DataFrame(results)

# --- Heatmap korelasi (Spearman) ---
plt.figure(figsize=(10, 8))
sns.heatmap(data[numcols].corr(method='spearman'), annot=True, cmap='coolwarm')
plt.title("Spearman Correlation Heatmap")
plt.show()

results_df

Menurut uji statistik, variable yang pengaruh adalah `edu_level, workload, stress, sleep_hours, work_env dan wlb (work-life balance)`

#### **Avg Satisfaction**

In [ ]:
avg_satisfaction = df.groupby('dept')['job_satisfaction'].mean().reset_index()
avg_satisfaction

In [ ]:
company_avg = df['job_satisfaction'].mean()

avg_satisfaction['satisfaction_value'] = avg_satisfaction['job_satisfaction'].apply(
    lambda x: 'Tinggi' if x > company_avg else 'Rendah'
)
print(company_avg)
avg_satisfaction

In [ ]:
# data = data.drop(columns=['satisfaction_segment'])

In [ ]:
# data = data.merge(avg_satisfaction[['dept', 'satisfaction_segment']], on='dept')
# data.head()

#### **Satisfaction per Segment per Employee**

In [ ]:
mean_js = data['job_satisfaction'].mean()
data['emp_satisfaction'] = data['job_satisfaction'].apply(
    lambda x: 'Tinggi' if x > mean_js else 'Rendah'
)

print(data['emp_satisfaction'].value_counts())

sns.boxplot(x='dept', y='job_satisfaction', data=data)
plt.title("Distribusi Job Satisfaction per Departemen")
plt.xticks(rotation=45)
plt.show()

data[['dept', 'job_satisfaction', 'emp_satisfaction']].head()

In [ ]:
# data = data.drop(columns=['satisfaction_segment'])
data.head()

In [ ]:
# 1. Histogram Job Satisfaction
plt.figure(figsize=(8,5))
sns.histplot(data['job_satisfaction'], bins=10, kde=True, color='skyblue')
plt.axvline(mean_js, color='red', linestyle='--', label=f'Mean: {mean_js:.2f}')
plt.title("Distribusi Job Satisfaction")
plt.xlabel("Job Satisfaction")
plt.ylabel("Frekuensi")
plt.legend()
plt.show()

# 2. Boxplot keseluruhan
plt.figure(figsize=(6,4))
sns.boxplot(y='job_satisfaction', data=data, color='lightgreen')
plt.title("Boxplot Job Satisfaction")
plt.ylabel("Job Satisfaction")
plt.show()

# 3. Countplot Segmen Kepuasan
plt.figure(figsize=(6,4))
sns.countplot(x='emp_satisfaction', data=data, palette='Set2')
plt.title("Jumlah Karyawan per Segmen Kepuasan")
plt.xlabel("Segmen Kepuasan")
plt.ylabel("Jumlah Karyawan")
plt.show()

### **Identifikasi segmen karyawan yang memiliki tingkat kepuasan lebih tinggi atau lebih rendah dibandingkan rata-rata.**

In [ ]:
# Hitung jumlah "Tinggi" dan "Rendah" tiap departemen
cross_tab = pd.crosstab(
    index=data['dept'],
    columns=data['emp_satisfaction'],
    margins=True,  # Menambahkan total
    margins_name="Total"
)
cross_tab

In [ ]:
# Hitung persentase "Tinggi" tiap departemen
percentage = data.groupby('dept')['emp_satisfaction'].apply(
    lambda x: (x == 'Tinggi').mean() * 100
).reset_index(name='% Tinggi')

percentage.sort_values(by='% Tinggi', ascending=False)

In [ ]:
# Hitung persentase 'Tinggi' dan 'Rendah' per dept
dept_summary = (
    data.groupby(['dept', 'emp_satisfaction'])
    .size()
    .unstack()
    .assign(
        Total=lambda x: x['Tinggi'] + x['Rendah'],
        Persen_Tinggi=lambda x: round((x['Tinggi'] / x['Total']) * 100, 2),
        Persen_Rendah=lambda x: round((x['Rendah'] / x['Total']) * 100, 2)
    )
)
print(mean_js)
dept_summary

In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(
    data=data,
    x='dept',
    hue='emp_satisfaction',
    order=data['dept'].value_counts().index
)
plt.title("Distribusi Tinggi/Rendah Job Satisfaction per Departemen")
plt.xticks(rotation=45)
plt.legend(title='Kepuasan')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
ax = sns.barplot(
    data=dept_summary.reset_index(),
    x='dept',
    y='Persen_Tinggi',
    color='skyblue',
    order=dept_summary.sort_values('Persen_Tinggi', ascending=False).index
)

# Add two-line labels
for i, p in enumerate(ax.patches):
    dept = dept_summary.index[i]
    count = dept_summary.loc[dept, 'Tinggi']
    total = dept_summary.loc[dept, 'Total']
    percentage = dept_summary.loc[dept, 'Persen_Tinggi']

    # First line (percentage)
    ax.annotate(
        f'{percentage:.1f}%',
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='center',
        xytext=(0, 5),
        textcoords='offset points',
        fontsize=10
    )

    # Second line (count)
    ax.annotate(
        f'({count}/{total})',
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='center',
        xytext=(0, -10),  # Position below the percentage
        textcoords='offset points',
        fontsize=9
    )

# Style adjustments
# plt.axhline(y=mean_js*20, color='red', linestyle='--', label='Rata-Rata Perusahaan')
plt.title('Persentase Karyawan dengan Kepuasan Tinggi per Departemen (Low/High)', pad=20)
plt.xlabel('Departemen')
plt.ylabel('Persentase Tinggi (%)')
plt.xticks(rotation=45)
plt.ylim(0, 110)  # Extra space for two-line labels
plt.legend()
plt.tight_layout()
plt.show()